In [ ]:
from matplotlib import font_manager, rc
from matplotlib.pyplot import savefig
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from pandas import DataFrame
from pandas.io import sql
import pandas as pd
import numpy as np

%matplotlib inline

font_name = font_manager.FontProperties(fname = 'C:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family = font_name, size=14)

## 연령별 전체 판매 수량

In [ ]:
cellData = pd.read_csv('', encoding='utf-8')
cellData.columns = ['buyDate', 'nowAge', 'pastAge', 'Location', 'consumerType', 'orderCount']
cellData.buyDate = cellData.buyDate.astype(str)

cellData['Year'] = cellData.buyDate.str[0:4]
cellData['Month'] = cellData.buyDate.str[4:6]

def Age_cate(x):
    if x<=20:
        return '20대 이하'
    if x<=30:
        return '20대'
    if x<=40:
        return '30대'
    if x<=50:
        return '40대'
    if x<=60:
        return '50대'
    if x<=70:
        return '60대'
    if x<=80:
        return '70대'
    if x<=90:
        return '80대'

cellData['Age_cate']=cellData['pastAge'].apply(Age_cate)
cellData['Do'] = cellData.Location.str[0:2]

del cellData['buyDate']
del cellData['nowAge']
del cellData['pastAge']

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
cellData.Age_cate.value_counts().plot(kind='bar', figsize=(16,10), title="전체 연령별(구매나이기준) 공청기 판매수량", table=True, ax=ax, fontsize=15)
ax.get_xaxis().set_visible(False)
ax.xaxis.set_label_text("연령대")
ax.yaxis.set_label_text("판매수량")

## 년도 지역별 전체 판매 수량

### 도기준

In [ ]:
cellData = pd.read_csv('', encoding='utf-8')
cellData.columns = ['buyDate', 'nowAge', 'pastAge', 'Location', 'consumerType', 'orderCount']
cellData.buyDate = cellData.buyDate.astype(str)

cellData['Year'] = cellData.buyDate.str[0:4]
cellData['Month'] = cellData.buyDate.str[4:6]

cellData['Do'] = cellData.Location.str[0:2]

cellData=cellData[cellData.consumerType.isin(['C'])]
cellData=pd.DataFrame(cellData.groupby(['Do','consumerType'])['orderCount'].sum())
cellData=cellData.reset_index()
cellData=cellData.sort(['orderCount'], ascending=[0])

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
cellData.head(15).plot(kind='bar', x='Do', ax=ax, rot=70, title="C타입을 가장 많이 구매한 지역", color='red', alpha=0.7)
ax.set_xlabel("지역")
ax.set_ylabel("판매수량")
ax.legend(["판매대수"])

### 지역기준

In [ ]:
cellData = pd.read_csv('', encoding='utf-8')
cellData.columns = ['buyDate', 'nowAge', 'pastAge', 'Location', 'consumerType', 'orderCount']
cellData.buyDate = cellData.buyDate.astype(str)

cellData['Year'] = cellData.buyDate.str[0:4]
cellData['Month'] = cellData.buyDate.str[4:6]

cellData=cellData[cellData.consumerType.isin(['C'])]
cellData=pd.DataFrame(cellData.groupby(['Location','consumerType'])['orderCount'].sum())
cellData=cellData.reset_index()
cellData=cellData.sort(['orderCount'], ascending=[0])

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
cellData.head(15).plot(kind='bar', x='Location', ax=ax, rot=70, title="C타입을 가장 많이 구매한 지역", color='red', alpha=0.7)
ax.set_xlabel("지역")
ax.set_ylabel("판매수량")
ax.legend(["판매대수"])

## 월별 지역의 최다 판매수량

In [ ]:
cellData = pd.read_csv('', encoding='utf-8')
cellData.columns = ['buyDate', 'nowAge', 'pastAge', 'Location', 'consumerType', 'orderCount']
cellData.buyDate = cellData.buyDate.astype(str)

cellData['Year'] = cellData.buyDate.str[0:4]
cellData['Month'] = cellData.buyDate.str[4:6]

def Age_cate(x):
    if x<=20:
        return '20대 이하'
    if x<=30:
        return '20대'
    if x<=40:
        return '30대'
    if x<=50:
        return '40대'
    if x<=60:
        return '50대'
    if x<=70:
        return '60대'
    if x<=80:
        return '70대'
    if x<=90:
        return '80대'

cellData['Age_cate']=cellData['pastAge'].apply(Age_cate)

cellData['Do'] = cellData.Location.str[0:2]
cellData['Si'] = cellData.Location.str[3:]
del cellData['Location']
del cellData['buyDate']
del cellData['nowAge']
del cellData['pastAge']

''' 타입 및 년도, 연령대 입력 '''
cellData=cellData[cellData.Year.isin(['2016']) & cellData.consumerType.isin(['A']) & cellData.Age_cate.isin(['30대'])]
total_cellData=pd.DataFrame(cellData.groupby(['Year','Month','Do','Si','consumerType','Age_cate'])['orderCount'].sum())

total_cellData=total_cellData.sort(['orderCount'], ascending=[0])
total_cellData=total_cellData.reset_index()
total_cellData=total_cellData.head(15)

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
ax2 = ax.twinx()

total_cellData['Dosi']=total_cellData['Do']+total_cellData['Si']

total_cellData.plot(x='Dosi', kind='bar', ax=ax, title="2016년 30대가 A타입을 많이 구매한 지역")
ax.set_ylabel("판매수량")
ax.set_xlabel("최다 판매 지역")
ax.set_ylim(0,total_cellData.orderCount.max()+2)

ax2.plot(total_cellData['Month'], linestyle='-', marker='o', linewidth=2.0, color='red')
ax2.set_ylabel("월별")
ax2.set_ylim(0,13)
ax2.set_yticks(list(range(1,13)))

# 지역별 판매 데이터 수치 분석

In [ ]:
'''2006년 부터 PM10 월별 수치 수집'''
dbData= pd.read_pickle('../data/공기청정기 구매데이터 PM10.pickle')

dbData.columns = ['Location', 'Date', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
dbData = dbData.sort_values(by = ['Location', 'Date'])

dbData = DataFrame(dbData, columns = ['Date', 'Location', 'PM10'])

''' dbData Preprocessing '''

dbData.Date = dbData.Date.astype(str)

dbData['Year'] = dbData.Date.str[0:4]
dbData['Month'] = dbData.Date.str[5:7]
dbData['Si'] = dbData.Location.str[0:2]

dbData = DataFrame(dbData, columns = ['Year', 'Month', 'Si', 'PM10'])

#print(dbData[dbData.Si.isin(['충남']) & dbData.Year.isin(['2006'])])

''' 판매데이터 read csv '''
airData = pd.read_csv('', encoding='utf-8')
airData.columns = ['buyDate', 'nowAge', 'pastAge', 'Location', 'consumerType', 'orderCount']

#2006.01 ~ 2016.03까지 수집
airData = airData.ix[airData['buyDate'] <= 201603]

#구매날짜 datatype 변경 object -> string
airData.buyDate = airData.buyDate.astype(str)

airData['Year'] = airData.buyDate.str[0:4]
airData['Month'] = airData.buyDate.str[4:6]

#구매한 지역 세부분 분리
airData['Si'] = airData.Location.str[0:2]

airData = pd.DataFrame(airData, columns = ['Year', 'Month', 'Location','Si', 'consumerType', 'orderCount'])

'''
해당 년월에 타입이 안들어 간 경우가 있으므로 dummyData를 통한 통합이 필요
'''
dummy_data = pd.DataFrame(dbData, columns=['Year','Month','Si'])
dummy_data['tmp']=1

df_dummy = pd.DataFrame({'consumerType':['A','B','C']})
df_dummy['tmp']=1
dummy_data=pd.merge(dummy_data, df_dummy, on='tmp').drop('tmp',axis=1)
dummy_data['orderCount']=0
'''
dummy data를 통해 전체 data merge
'''
total_merge=pd.concat([dummy_data, airData], ignore_index=True)
total_merge=pd.DataFrame(total_merge.groupby(['Year','Month','Location','Si','consumerType'])['orderCount'].sum())
total_merge=total_merge.reset_index()

#total_merge=pd.merge(total_merge, dbData, on=['Year','Month','Location','Si'], how='left')
total_merge=pd.merge(total_merge, dbData, on=['Year','Month','Si'], how='left')

'''
단일 지역 시각화그리기

지역과 년도 input
'''
location_single=total_merge[total_merge.Si.isin(['경기']) & total_merge.Year.isin(['2012'])]
PM10_test = location_single.groupby(['Year','Month','Si'])['PM10'].mean()
PM10_v=PM10_test.values

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
ax2 = ax.twinx()

location_single = location_single.groupby(['Year','Month','Si','consumerType'])['orderCount'].sum().unstack(['consumerType'])
location_single = location_single.reset_index()
print(location_single.head())
location_single.plot(x='Month',kind='bar', ax=ax, figsize=(16,10), use_index=True, rot=0, alpha=0.75, \
                     title="2012년 경기지역의 구매량 및 미세먼지 수치")

ax.xaxis.set_label_text("월")
ax.yaxis.set_label_text("판매수량")
ax.tick_params(labelsize=20)
ax2.plot(PM10_v, linestyle='-', marker='o', linewidth=2.0, color='black')
ax2.set_ylabel("미세먼지 수치")

In [ ]:
'''
여러 지역 시각화 그리기

강원 제주지역 예시
'''
location_multi=total_merge[total_merge.Si.isin(['강원','제주']) & total_merge.Year.isin(['2016'])]
PM10_multi = location_multi.groupby(['Year','Month'])['PM10'].mean()
PM10_multi=PM10_multi.values

fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(111)
ax2 = ax.twinx()

location_multi=location_multi.groupby(['Year','Month','consumerType'])['orderCount'].sum().unstack(['consumerType'])
location_multi=location_multi.reset_index()

#test2.plot(x='Month',kind='bar', ax=ax, figsize=(16,10), use_index=True, rot='vertical', subplots=True)
location_multi.plot(x='Month',kind='bar', ax=ax, figsize=(16,10), use_index=True, rot=0, alpha=0.75,title="2016년도의 강원, 제주지역의 공기청정기 구매량")

ax.xaxis.set_label_text("월")
ax.yaxis.set_label_text("판매수량")
ax2.plot(PM10_multi, linestyle='-', marker='o', linewidth=2.0, color='black')
ax2.set_ylabel("미세먼지 수치")